In [ ]:
!pip install -U datasets huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

In [ ]:
# Download the NLTK sentence tokenizer data if you haven't already
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
squad_dataset = load_dataset('squad_v2')
squad_df = pd.concat([squad_dataset['train'].to_pandas(), squad_dataset['validation'].to_pandas()], ignore_index=True)
squad_df = squad_df.drop(["question", "answers", "title", "id"], axis=1)
squad_df = squad_df.drop_duplicates(subset=['context'])
squad_df['text'] = squad_df['context'].apply(lambda text: sent_tokenize(text))
squad_df = squad_df.explode('text').reset_index(drop=True)
squad_df = squad_df.drop(["context"], axis=1)
squad_df = squad_df[squad_df['text'].apply(lambda x: len(x) >= 10)]
squad_df['malicious'] = False
print(squad_df.columns)

Index(['text', 'malicious'], dtype='object')


In [ ]:
hackaprompt_dataset = load_dataset("jerpint-org/HackAPrompt-Playground-Submissions")
hackaprompt_df = hackaprompt_dataset['train'].to_pandas()
hackaprompt_df = hackaprompt_df[hackaprompt_df['level'] <= 8]
hackaprompt_df = hackaprompt_df[hackaprompt_df['token_count'] >= 10]
hackaprompt_df = hackaprompt_df.drop(["prompt", "expected_completion", "model", "token_count", "correct", "error", "score", "timestamp", "session_id", "level", "completion"], axis=1)
hackaprompt_df = hackaprompt_df.drop_duplicates(subset=['user_input'])
hackaprompt_df["text"] = hackaprompt_df['user_input']
hackaprompt_df = hackaprompt_df.drop(['user_input'], axis=1)
hackaprompt_df['malicious'] = True
print(hackaprompt_df.columns)

Index(['text', 'malicious'], dtype='object')


In [ ]:
print(len(hackaprompt_df), len(squad_df))
df = pd.concat([hackaprompt_df, squad_df], axis=0, ignore_index=True)

296224 100215


In [ ]:
csv_file = 'pi_data.csv'
df.to_csv(csv_file, index=False)
df.head(-5)
hf_dataset = Dataset.from_pandas(df)
print(hf_dataset)
hf_dataset.push_to_hub("reinforz/pi_hackaprompt_squad")

Dataset({
    features: ['text', 'malicious'],
    num_rows: 396439
})


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/397 [00:00<?, ?ba/s]